In [216]:
#imports
from keras.initializers import glorot_uniform
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import datasets, layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np

In [217]:
#generate data
x_train = np.random.random((20,8)) 
y_train = np.random.randint(2, size = (20, 1))
x_test = np.random.random((20,8))
y_test = np.random.randint(2, size=(20,1))

In [218]:
#functions
def getWeights(particle):
    model.set_weights(particle)
    #get weights
    d16x, d16w = model.layers[0].get_weights()
    d32x, d32w = model.layers[1].get_weights()
    d1x, d1w = model.layers[2].get_weights()
    return d16x, d32x, d1x

def printParticle(particle, fitness ):
    model.set_weights(particle)
    d16x, d32x, d1x = getWeights(particle)
    print("Accuracy: ", fitness)
    print("Dense Layer 16")
    print(d16x.shape)
    print(d16x)
    print()
    print("Dense Layer 32")
    print(d32x.shape)
    print(d32x)
    print()
    print("Dense Layer 1")
    print(d1x.shape)
    print(d1x)
    print()
    
def buildParticle():
    initial_weights = model.get_weights()
    k_eval = lambda placeholder: placeholder.numpy()
    new_weights = [k_eval(glorot_uniform()(w.shape)) for w in initial_weights]
    model.set_weights(new_weights)
    #find fitness
    fitness = model.evaluate(x_test, y_test, batch_size=128)
    return new_weights, fitness

def printParticlesFit(pFits):
    for x in range(0, 3):
        print("Particle", x, ": fitness", pFits[x])
    print("Fittest Particle:", leadInd, "at", leadFit)

def findFittest(pFits):
    leadFit = 0
    for par in range(0, 3):
        if leadFit < pFits[par]:
            leadFit = pFits[par]
            leadInd = par
    return leadFit, leadInd

def moveParticles(particles, pFits):
    for par in range(0,3):
        if par == leadInd:
            continue
        moveParticle(particles[par][0], particles[leadInd][0], par)
        
def moveParticle(particle, leadParticle, par):
    #calculate movement to lead
    leadChange = 0.03
    px, py, pz = getWeights(particle)
    lx, ly, lz = getWeights(leadParticle)
    dx = (lx - px)* leadChange
    dy = (ly - py)* leadChange
    dz = (lz - pz)* leadChange
    #calculate self-driven movement
    sx, sy, sz = findDirection(particle, par)
    dx,dy,dz = dx+sx, dy+sy, dz+sz
    px, py, pz = px + dx, py + dy, pz + dz
    a,b = model.layers[0].get_weights()
    c,d = model.layers[1].get_weights()
    e,f = model.layers[2].get_weights()
    model.layers[0].set_weights([px,b])
    model.layers[1].set_weights([py,d])
    model.layers[2].set_weights([pz,f])
    #model.set_weights([[px,b],[py,d],[pz,f]])
    fitness = model.evaluate(x_test, y_test, batch_size=128)
    pFits[par] = fitness
    particles[par][0] = model.get_weights()
    
def findDirection(particle, ind):
    iChg = 0.02
    px, py, pz = getWeights(particle)
    a,b = model.layers[0].get_weights()
    c,d = model.layers[1].get_weights()
    e,f = model.layers[2].get_weights()
    #easy way to copy containers
    mx, my, mz = px+iChg, py+iChg, pz+iChg
    model.layers[0].set_weights([mx,b])
    fitness = model.evaluate(x_test, y_test, batch_size=128)
    if(fitness > pFits[ind]):
        px = mx
    model.layers[1].set_weights([my,d])
    fitness = model.evaluate(x_test, y_test, batch_size=128)
    if(fitness > pFits[ind]):
        py = my
    model.layers[2].set_weights([mz,f])
    fitness = model.evaluate(x_test, y_test, batch_size=128)
    if(fitness > pFits[ind]):
        pz = mz
    return px, py, pz

In [219]:
#model
model = models.Sequential()
model.add(layers.Dense(16, kernel_initializer = 'random_uniform', input_dim = 8, activation = 'sigmoid'))
model.add(layers.Dense(32, activation = 'sigmoid'))
model.add(layers.Dense(1,activation='sigmoid'))
#compile
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metric = ['accuracy'])
#find fitness
fitness = model.evaluate(x_test, y_test, batch_size=128)

20/1 [========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 4ms/sample - loss: 0.9650


In [220]:
##base step setup
#particle instantiation (3)
particle1 = model.get_weights()
fitness1 = fitness

particle2, fitness2 = buildParticle()
particle3, fitness3 = buildParticle()

#set parameters of new paritcles
particles = np.array([[particle1], [particle2], [particle3]])
pFits = np.array([[fitness1],[fitness2],[fitness3]])

#fitness instantiation
leadFit, leadInd = findFittest(pFits)

20/1 [========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 160us/sample - loss: 0.6715


In [221]:
printParticlesFit(pFits)

Particle 0 : fitness [0.96499902]
Particle 1 : fitness [0.87616539]
Particle 2 : fitness [0.67152989]
Fittest Particle: 0 at [0.96499902]


In [215]:
#n step setup
#calculate movement
steps = 3
for par in range(0,steps):
    moveParticles(particles, pFits)
    leadFit, leadInd = findFittest(pFits)
    printParticlesFit(pFits)


20/1 [========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 158us/sample - loss: 3.6540
Particle 0 : fitness [2.80634141]
Particle 1 : fitness [3.65400696]
Particle 2 : fitness [1.06834483]
Fittest Particle: 1 at [3.65400696]
20/1 [=======================================================================================================================================================================================================================